In [10]:
# from dodfminer import ContentExtractor
# from dodfminer import ActsExtractor
# from dodfminer import downloader
# from dodfminer.downloader.core import Downloader
import pandas as pd
import numpy as np
import glob
import re
import os

# Dataframe

In [11]:
data = pd.read_pickle('acts/abertura.pkl')

In [12]:
data

,Tipo do Ato,Id Ato,DODF,Data Ato/DODF,Processo GDF/SEI,Tipo,Modo de Disputa,Objeto,Valor Estimado,Texto,Redator
0,AVISO DE ABERTURA DE LICITACAO,NaN,2021\01_Janeiro\DODF 003 06-01-2021 1,05 de janeiro de 2021,0005000011038202083,Menor Preco,Aberto,NaN,NaN,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,NaN
1,AVISO DE ABERTURA DE LICITACAO,NaN,2021\01_Janeiro\DODF 005 08-01-2021 1,07 de janeiro de 2021,0005000032913202061,Menor Preco,Aberto,NaN,\nSigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,NaN
2,AVISO DE ABERTURA DE LICITACAO,NaN,2021\01_Janeiro\DODF 008 13-01-2021 1,13-01-2021,0005000005354202016,Menor Preco,Aberto,NaN,Sigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,NaN
3,AVISO DE ABERTURA DE LICITACAO,NaN,2021\01_Janeiro\DODF 009 14-01-2021 1,14-01-2021,0006300004536202058,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO No 01/2021 - UA...,NaN
4,AVISO DE ABERTURA DE LICITACAO,NaN,2021\01_Janeiro\DODF 014 21-01-2021 1,21-01-2021,0402600022953202025,Menor Preco,NaN,NaN,Sigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
113,AVISO DE ABERTURA DE LICITACAO,NaN,2021\07_Julho\DODF 126 07-07-2021 1,07-07-2021,0006300001472202114,NaN,NaN,NaN,"de R$ 161.112,96",AVISO DE ABERTURA DE LICITACAO No 18/2021 - UA...,NaN
114,AVISO DE ABERTURA DE LICITACAO,NaN,2021\07_Julho\DODF 127 08-07-2021 1,06 de julho de 2021,0005000032720202018,Menor Preco,Aberto,NaN,R$ SIGILOSO,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,NaN
115,AVISO DE ABERTURA DE LICITACAO,NaN,2021\07_Julho\DODF 131 14-07-2021 1,09 de julho de 2021,0011100000528202118,Menor Preco\n<END_OF_BLOCK>\nObjeto:\n<END_OF...,NaN,NaN,,AVISO DE ABERTURA DE LICITACAO O Presidente da...,NaN
116,AVISO DE ABERTURA DE LICITACAO,NaN,2021\07_Julho\DODF 133 16-07-2021 1,16-07-2021,00402600015845202187,Menor Preco,NaN,NaN,Sigiloso,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,NaN


In [25]:
for row in range(len(data)):
    if data['Valor Estimado'][row] == ' ' or data['Valor Estimado'][row] == '\n':
        data['Valor Estimado'][row] = np.nan

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Preprocess

In [13]:
import os
import re
import nltk
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin


DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


class CRFFeaturizer(BaseEstimator, TransformerMixin):

    @classmethod
    def crf_sentence_featurizer(cls, tokens: Iter[str]):
        """Generates features from sequence of tokens.

        Args:
            tokens (Iter[str]): [description]

        Returns:
            List[Dict]: list of dicts containing the feature of each token
        """
        return [{
                'word': token.lower(),
                'capital_letter': token[0].isupper(),
                'all_capital': token.isupper(),
                'isdigit': token.isdigit(),
                'word_before': token.lower() if j==0 else tokens[j-1].lower(),
                'word_after:': token.lower() if j+1>=len(tokens) else tokens[j+1].lower(),
                'BOS': j==0,
                'EOS': j==len(tokens)-1
            }
            for j, token in enumerate(tokens) if token
        ]



    def __init__(self, sent_featurizer: Callable = None,):
        self.text_matrix  = []
        self.sent_featurizer = sent_featurizer or self.crf_sentence_featurizer


    def fit(self, X = None, y=None, **fit_params):
        return self


    def transform(self, X: Iter[Iter[str]] ):
        if not isinstance(X, pd.Series):
            X = pd.Series(X)
        X = X.map(self.sent_featurizer)
        return X.values

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


# Dataframe com as entidade selecionadas

In [26]:
df = data[['Tipo do Ato', 'Data Ato/DODF', 'Processo GDF/SEI', 'Tipo', 'Modo de Disputa', 'Valor Estimado', 'Texto']]

df.rename({'Tipo do Ato': 'ATO', 'Data Ato/DODF': 'DATA', 'Processo GDF/SEI': 'PROCESSO', 'Tipo': 'TIPO', 'Modo de Disputa': 'DISPUTA', 'Valor Estimado': 'V_ESTIMADO'}, axis=1, inplace=True)

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# IOBifyer

In [27]:
iob = IOBifyer(column='Texto')
r = iob.transform(df)

In [28]:
r

0      [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1      [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
2      [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
3      [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
4      [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
                             ...                        
113    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
114    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
115    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
116    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
117    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
Length: 118, dtype: object

In [29]:
df.shape, r.shape

((118, 7), (118,))

In [30]:
df["IOB"] = np.nan

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
for idx in range(len(df)):
    df["IOB"][idx] = ' '.join(r[idx])

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [32]:
df

,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,Texto,IOB
0,AVISO DE ABERTURA DE LICITACAO,05 de janeiro de 2021,0005000011038202083,Menor Preco,Aberto,NaN,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1,AVISO DE ABERTURA DE LICITACAO,07 de janeiro de 2021,0005000032913202061,Menor Preco,Aberto,\nSigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
2,AVISO DE ABERTURA DE LICITACAO,13-01-2021,0005000005354202016,Menor Preco,Aberto,Sigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
3,AVISO DE ABERTURA DE LICITACAO,14-01-2021,0006300004536202058,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO No 01/2021 - UA...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
4,AVISO DE ABERTURA DE LICITACAO,21-01-2021,0402600022953202025,Menor Preco,NaN,Sigiloso,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
...,...,...,...,...,...,...,...,...
113,AVISO DE ABERTURA DE LICITACAO,07-07-2021,0006300001472202114,NaN,NaN,"de R$ 161.112,96",AVISO DE ABERTURA DE LICITACAO No 18/2021 - UA...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
114,AVISO DE ABERTURA DE LICITACAO,06 de julho de 2021,0005000032720202018,Menor Preco,Aberto,R$ SIGILOSO,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
115,AVISO DE ABERTURA DE LICITACAO,09 de julho de 2021,0011100000528202118,Menor Preco\n<END_OF_BLOCK>\nObjeto:\n<END_OF...,NaN,NaN,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
116,AVISO DE ABERTURA DE LICITACAO,16-07-2021,00402600015845202187,Menor Preco,NaN,Sigiloso,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...


In [33]:
df['Texto'][0]

'AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONICO No 55/2020 Processo: 0005000011038202083. TIPO: Menor Preco. MODO DE DISPUTA: Aberto. OBJETO: Aquisicao de bens de imagem, iluminacao e sonorizacao, visando atender demanda da Secretaria de Estado de Seguranca Publica do Distrito Federal. VALOR ESTIMADO: Sigiloso. PRAZOS: De entrega: 30 (trinta) dias corridos contados do recebimento da nota de empenho ou da assinatura do contrato. Do Contrato: 90 (noventa) dias a partir da data da assinatura. DATA DA SESSAO PUBLICA: 18/01/2021 as 09:00 horas no www.gov.br/compras. UASG 450107. Edital tambem esta disponivel no http://www.ssp.df.gov.br/licitacoes/. Brasilia/DF, 05 de janeiro de 2021 FERNANDO CESAR NEVES '

In [34]:
len(DEFAULT_TOKENIZER(df['Texto'][0]))

114

In [35]:
df['IOB'][0]

'B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PROCESSO O B-TIPO I-TIPO O O O B-DISPUTA O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-DATA I-DATA I-DATA I-DATA I-DATA O O O'

In [36]:
len(df['IOB'][0].split())

114